# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.append('../../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from model_evaluator import DataSource, DataPredSelection
from base_models import BaseModel
from optimizers import Optimizer
from model_creator import MTLApproach, NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'train_vgg16',
        'description': 'Training mtl network for ALL requisites - 10 epochs - without early_stopping in FVC GT dataset NOT aligned',
        'tags': ['vgg16', 'ground truths', 'adamax', 'mtl', '10 epochs'],
        'src_files': ["src/**/*.py"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_1,
        'reqs': list(cts.ICAO_REQ),
        'aligned': True,
        'use_gt_data': True,
        'gt_names': {
            'train_validation': [],
            'test': [],
            'train_validation_test': [GTName.FVC]
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': False,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 32,
        'n_epochs': 10,
        'early_stopping': 10,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 3,
        'max_train_steps_per_op': 5,
        'n_trials': 2
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 16240
Process ID: 19983
---------------------------
-----
Use Neptune:  False
-----
-------------------
Args: 
{'exp_params': {'description': 'Training mtl network for ALL requisites - 10 '
                               'epochs - without early_stopping in FVC GT '
                               'dataset NOT aligned',
                'name': 'train_vgg16',
                'src_files': ['src/**/*.py'],
                'tags': ['vgg16',
                         'ground truths',
                         'adamax',
                         'mtl',
                         '10 epochs']},
 'nas_params': {'max_blocks_per_branch': 3,
                'max_train_steps_per_op': 5,
                'n_trials': 2},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f8a47f023a0>}>,
                      'batch_size

# Load Data

In [16]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4926, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


# Producing Fake Data

In [18]:
runner.produce_fake_data()

-------------------- producing fake data for experimental purposes -------------------
fake_train_data.shape: (500, 26)
fake_validation_data_df.shape: (100, 26)
fake_test_data_df.shape: (50, 26)
fake_train_data.mouth.sum: 266.0
fake_train_data.veil.sum: 242.0
fake_train_data.flash_lenses.sum: 264.0
fake_train_data.hat.sum: 245.0
fake_train_data.backgrouund.sum: 249.0


# Data Generators

In [6]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 500 validated image filenames.
Found 100 validated image filenames.
Found 50 validated image filenames.
TOTAL: 650

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Neural Architecture Search

In [7]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
++++++++++++++++++++++++++++++++++++++++++++++++++ STARTING NEW TRAIN ++++++++++++++++++++++++++++++++++++++++++++++++++
 ----- Training 0 | Config: {'n_denses_0': 4, 'n_denses_1': 5, 'n_denses_2': 3, 'n_denses_3': 5} --------
Creating model...
Model created
Training VGG16 network
 .. Not fine tuning base model...
  .. Total params: 15,759,854
  .. Trainable params: 1,045,166
  .. Non-trainable params: 14,714,688
Epoch 1/2
15/15 [==============================] - 22s 693ms/step - loss: 1.8650 - background_loss: 0.8015 - close_loss: 0.7622 - ink_mark_loss: 0.7461 - pixelation_loss: 0.7320 - washed_out_loss: 1.2136 - blurred_loss: 0.7453 - sh_head_loss: 0.8606 - mouth_loss: 0.7157 - veil_loss: 0.7385 - red_eyes_loss: 0.9083 - flash_lenses_loss: 0.8802 - dark_glasses_loss: 0.8728 - l_away_loss: 0.8978 - frame_eyes_loss: 0.8081 - hair_eyes_loss: 0.7620 - eyes_closed_loss: 0.8945 - frames_heavy_loss: 0.9037 - sh_face_lo

{'model_0': {'final_EER_mean': 50.91,
  'final_ACC': 49.09,
  'config': {'n_denses_0': 4,
   'n_denses_1': 5,
   'n_denses_2': 3,
   'n_denses_3': 5}},
 'model_1': {'final_EER_mean': 50.0,
  'final_ACC': 49.18,
  'config': {'n_denses_0': 3,
   'n_denses_1': 2,
   'n_denses_2': 2,
   'n_denses_3': 1}}}

# Testing Network Modification